- open main folder
- convert all scans to nii
- read xlsx
- select corresponding scan Number

- coregistration afni
- rel. cbv with 3dcalc

In [31]:
import os
import subprocess
import numpy as np
import pandas as pd
import MRIhelperfunctions as h
import MRIhelperfunctions
from imp import importlib
import fnmatch


In [79]:
importlib.reload(h)
import MRIhelperfunctions as h

In [81]:
main_folder = '/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata'

groupstatsfolder = os.path.join(file_path,'groupstats')


In [84]:
info = pd.read_excel(f"{main_folder}/Animal_Scan_IDs.xlsx")
info.columns = map(str.lower, info.columns)
# info = info.append({'animal-id': 'folder'}, ignore_index=True)



# def get_subject_folders(main_folder, info):
#     for animal in info.columns

info = h.find_animal_folders(info, main_folder)
info

# for folderName, subfolders, filenames in os.walk(main_folder):
#     for file in filenames:
#         if (fnmatch.fnmatch(file, '*.nii') and 'analyzed' not in folderName and 'groupstats' not in folderName):
#             ID = h.splitall(folderName)[-1]
#             print(ID)
#             file_path1 = glob.glob(os.path.join(folderName,file))
#             file_path2 = os.path.dirname(file_path1[0]) # folder
#             file_path1 = re.sub(r'\+orig.BRIK.*.*',r'',file_path1[0])  # path to brik file without +orig ending
#             nb_kwargs = {'file_path': file_path1, 'folderName': file_path2, 'group': True}
#             print('running analysis for: ', file_path1)

            
            
# if not os.path.exists(groupstatsfolder):
#     os.makedirs(groupstatsfolder)

,animal-id,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
0,Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
1,Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
2,Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
3,Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
4,Diamox,15,14,13,11,13,15,12,13,14,13,16,17,20,16,14,11,13
folder,NaN,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...


In [88]:
import nibabel as nib

# for animalpath in info.loc['folder','glut1_male1']:   
animalpath=info.loc['folder','glut1_male1']
print(animalpath)
gdDCE = os.path.join(animalpath, 'X6P1.nii')
img = nib.load(gdDCE)
#     h.convertAll(animalpath)
    
    

/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_101508_Glut1_2_2


In [89]:
import p